In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np 
import os
from scipy.io.wavfile import read,write
import random
from shutil import copyfile

In [ ]:
savepath = '/content/drive/MyDrive/DataScienceProjects/Musical/Data'
trainpath = '/content/drive/MyDrive/DataScienceProjects/Musical/Data/train'
testpath = '/content/drive/MyDrive/DataScienceProjects/Musical/Data/test'
instruments = ['flu', 'gac', 'gel', 'org', 'pia', 'sax', 'vio', 'voi']
noinst = 8 

In [ ]:
def break_into_parts(readpath,writepath,ins,id):
	(fs,audio) = read(readpath)
	t = np.arange(0,len(audio)-1)/fs
	winLen = 2 #length of each excerpt in seconds
	nFiles = np.floor(t[-1]/winLen)

	start = 0
	for n in range(int(nFiles)):
		excerpt = audio[start:start+(winLen*fs)]
		write(os.path.join(writepath,ins+str(id)+".wav"),fs,excerpt)
		start = start + winLen*fs 
		id += 1

	return id

In [ ]:
def split_into_train_and_test(ins_path, ins, numFiles=600):
	#split 80:20
	nTrain = int(numFiles * 0.80)
	trp = os.path.join(trainpath,ins)
	tsp = os.path.join(testpath,ins)

	for r,d,files in os.walk(ins_path):
		random.shuffle(files)
		train = files[:nTrain]
		test = files[nTrain:]
		for ft in train:
			copyfile(os.path.join(ins_path,ft),os.path.join(trp,ft))
		for ft in test:
			copyfile(os.path.join(ins_path,ft),os.path.join(tsp,ft))


In [ ]:
def main():
	songs = []
	if not os.path.exists(trainpath):
		os.mkdir(trainpath)
		os.mkdir(testpath)

	#get individual subdirectory for each song
	# r - root, d - directories, f - files
	for r,d,f in os.walk(savepath):
		for song in d:
			if(song[0].isdigit()):
				songs.append(os.path.join(r,song))

	#go through all the files in each song directory and get the wav files only
	sound_files = []
	for ins in instruments:
		id = 0
		for song in songs:
			# print(song)
			songname = song[song.rfind('/')+1:]
			# print(songname)
			for r,d,files in os.walk(song):
				for file in files:
					if(file.endswith(".wav") and ins in file):
						readPath = os.path.join(song,file)
						writePath = os.path.join(savepath,ins)
						if not os.path.exists(writePath):
							os.mkdir(writePath)
						id = break_into_parts(readPath,writePath,ins,id)
						print('Saved ' + file)

					
	#split excerpts into train and test files
	for ins in instruments:
		trpath = os.path.join(trainpath,ins)
		tspath = os.path.join(testpath,ins)
		if not os.path.exists(trpath):
			os.mkdir(trpath)
			os.mkdir(tspath)
		split_into_train_and_test(os.path.join(savepath,ins),ins)

In [ ]:
if __name__ == '__main__':
    main()